In [12]:
import re, json, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import lightgbm as lgb
from pandas.api.types import CategoricalDtype

In [82]:
df1 = pd.read_excel('../../data/train_l1.xlsx')

In [83]:
df1

,Công lập (1/0),Tỉnh/TP,Tổ hợp môn,Học phí,hsg_1,hsg_2,hsg_3,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,truong-nganh-ptxt,nhom_nganh
0,1,An Giang,NaN,14100000.0,Toán,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
1,1,An Giang,NaN,14100000.0,Lý,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
2,1,An Giang,NaN,14100000.0,Hoá,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
3,1,An Giang,NaN,14100000.0,Sinh,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
4,1,An Giang,NaN,14100000.0,Tin,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
...,...,...,...,...,...,...,...,...,...,...,...,...
40910,1,TP. Hồ Chí Minh,C01,62000000.0,0,0,0,0,0,1,SPK-7510303A-Ưu Tiên,751
40911,1,Bình Phước,C01,32000000.0,0,0,0,0,0,1,SPK-7510301BP-Ưu Tiên,751
40912,1,TP. Hồ Chí Minh,C02,32000000.0,0,0,0,0,0,1,SPK-7510401V-Ưu Tiên,751
40913,1,TP. Hồ Chí Minh,A01,49000000.0,0,0,0,0,0,1,SPK-7510205N-Ưu Tiên,751


In [5]:
mask = df["Học phí"].isna()

df.loc[mask, "Học phí"] = df.loc[mask, "truong-nganh-ptxt"].apply(
    lambda x: 0 if str(x).startswith("SPS") 
    else 32000000 if str(x).startswith("SPK") 
    else 60000000 if str(x).startswith("QSX") 
    else np.nan
)

In [6]:
df[df["Học phí"].isna()]["truong-nganh-ptxt"].unique()

array([], dtype=object)

In [8]:
rename_map = {
    "Công lập (1/0)": "cong_lap",
    "Tỉnh/TP": "tinh_tp",
    "Học phí": "hoc_phi",
    "hsg_1": "hsg_1",
    "hsg_2": "hsg_2",
    "hsg_3": "hsg_3",
    "ahld": "ahld",
    "dan_toc_thieu_so": "dan_toc_thieu_so",
    "haimuoi_huyen_ngheo_tnb": "haimuoi_huyen_ngheo_tnb",
    "truong-nganh-ptxt": "truong_nganh_ptxt",
    "nhom_nganh": "nhom_nganh",
    "Tổ hợp môn": "to_hop_mon",
}
df = df.rename(columns=rename_map)

In [11]:
print(df)

       cong_lap          tinh_tp to_hop_mon     hoc_phi hsg_1 hsg_2 hsg_3  \
0             1         An Giang        NaN  14100000.0  Toán     0     0   
1             1         An Giang        NaN  14100000.0    Lý     0     0   
2             1         An Giang        NaN  14100000.0   Hoá     0     0   
3             1         An Giang        NaN  14100000.0  Sinh     0     0   
4             1         An Giang        NaN  14100000.0   Tin     0     0   
...         ...              ...        ...         ...   ...   ...   ...   
40910         1  TP. Hồ Chí Minh        C01  62000000.0     0     0     0   
40911         1       Bình Phước        C01  32000000.0     0     0     0   
40912         1  TP. Hồ Chí Minh        C02  32000000.0     0     0     0   
40913         1  TP. Hồ Chí Minh        A01  49000000.0     0     0     0   
40914         1       Bình Phước        D01  32000000.0     0     0     0   

       ahld  dan_toc_thieu_so  haimuoi_huyen_ngheo_tnb  \
0         0      

In [13]:
df['cong_lap']  = pd.to_numeric(df['cong_lap'], errors='coerce').fillna(0).astype('int8')
df['hoc_phi']   = pd.to_numeric(df['hoc_phi'],  errors='coerce')
df['nhom_nganh']= pd.to_numeric(df['nhom_nganh'], errors='coerce').astype('Int64')

for c in ['tinh_tp','to_hop_mon','truong_nganh_ptxt']:
    df[c] = df[c].astype(str).replace({'nan':'UNK','None':'UNK'})

prio_cols = ['hsg_1','hsg_2','hsg_3','ahld','dan_toc_thieu_so','haimuoi_huyen_ngheo_tnb']
for c in prio_cols:
    df[c] = df[c].fillna(0)

def pick_hsg(row):
    for k in ['hsg_1','hsg_2','hsg_3']:
        v = row[k]
        if isinstance(v, str) and v != '0' and v.strip() != '':
            return v.strip()
        if isinstance(v, (int, float)) and v != 0:
            return str(v)
    return "0"

df['hsg_subject'] = df.apply(pick_hsg, axis=1)

In [14]:
df

,cong_lap,tinh_tp,to_hop_mon,hoc_phi,hsg_1,hsg_2,hsg_3,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,truong_nganh_ptxt,nhom_nganh,hsg_subject
0,1,An Giang,UNK,14100000.0,Toán,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Toán
1,1,An Giang,UNK,14100000.0,Lý,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Lý
2,1,An Giang,UNK,14100000.0,Hoá,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Hoá
3,1,An Giang,UNK,14100000.0,Sinh,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Sinh
4,1,An Giang,UNK,14100000.0,Tin,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Tin
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40910,1,TP. Hồ Chí Minh,C01,62000000.0,0,0,0,0,0,1,SPK-7510303A-Ưu Tiên,751,0
40911,1,Bình Phước,C01,32000000.0,0,0,0,0,0,1,SPK-7510301BP-Ưu Tiên,751,0
40912,1,TP. Hồ Chí Minh,C02,32000000.0,0,0,0,0,0,1,SPK-7510401V-Ưu Tiên,751,0
40913,1,TP. Hồ Chí Minh,A01,49000000.0,0,0,0,0,0,1,SPK-7510205N-Ưu Tiên,751,0


## Lightgbm (not good)

In [22]:
for c in ['ahld','dan_toc_thieu_so','haimuoi_huyen_ngheo_tnb']:
    df[c] = (df[c].astype(str) != '0').astype('int8')

df['prio_any'] = (
    (df['hsg_subject'] != '0') |
    (df['ahld'] == 1) |
    (df['dan_toc_thieu_so'] == 1) |
    (df['haimuoi_huyen_ngheo_tnb'] == 1)
).astype('int8')

In [16]:
label_col = 'truong_nganh_ptxt'
classes, y = np.unique(df[label_col].astype(str).values, return_inverse=True)
num_class = len(classes)
num_class

1191

In [31]:
cat_cols = [
    'cong_lap','tinh_tp','hsg_subject',
]
num_cols = [
    'hoc_phi','nhom_nganh','ahld','dan_toc_thieu_so','haimuoi_huyen_ngheo_tnb'
]

for c in cat_cols:
    df[c] = df[c].astype('category')
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

X = df[cat_cols + num_cols]

In [32]:
X

,cong_lap,tinh_tp,hsg_subject,hoc_phi,nhom_nganh,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb
0,1,An Giang,Toán,14100000.0,714,0,0,0
1,1,An Giang,Lý,14100000.0,714,0,0,0
2,1,An Giang,Hoá,14100000.0,714,0,0,0
3,1,An Giang,Sinh,14100000.0,714,0,0,0
4,1,An Giang,Tin,14100000.0,714,0,0,0
...,...,...,...,...,...,...,...,...
40910,1,TP. Hồ Chí Minh,0,62000000.0,751,0,0,1
40911,1,Bình Phước,0,32000000.0,751,0,0,1
40912,1,TP. Hồ Chí Minh,0,32000000.0,751,0,0,1
40913,1,TP. Hồ Chí Minh,0,49000000.0,751,0,0,1


In [33]:
def round_budget_series(s: pd.Series) -> pd.Series:
    x = pd.to_numeric(s, errors='coerce').fillna(0).astype(float)
    bins   = [-np.inf, 0, 10e6, 20e6, 40e6, 60e6, 80e6, 100e6, 200e6, np.inf]
    labels = [0,        10e6,  20e6,  40e6,  60e6,  80e6,  100e6,  200e6, 500e6]
    return pd.cut(x, bins=bins, labels=labels, right=True, include_lowest=True).astype(float).astype('int64')

data = X.copy()
data['hoc_phi_ceiled'] = round_budget_series(data['hoc_phi']).astype('int64')

In [81]:
data

,cong_lap,tinh_tp,hsg_subject,hoc_phi,nhom_nganh,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,hoc_phi_ceiled,fee_bin_idx,is_free
0,1,An Giang,Toán,14100000.0,714,0,0,0,20000000,2,0
1,1,An Giang,Lý,14100000.0,714,0,0,0,20000000,2,0
2,1,An Giang,Hoá,14100000.0,714,0,0,0,20000000,2,0
3,1,An Giang,Sinh,14100000.0,714,0,0,0,20000000,2,0
4,1,An Giang,Tin,14100000.0,714,0,0,0,20000000,2,0
...,...,...,...,...,...,...,...,...,...,...,...
40910,1,TP. Hồ Chí Minh,0,62000000.0,751,0,0,1,80000000,5,0
40911,1,Bình Phước,0,32000000.0,751,0,0,1,40000000,3,0
40912,1,TP. Hồ Chí Minh,0,32000000.0,751,0,0,1,40000000,3,0
40913,1,TP. Hồ Chí Minh,0,49000000.0,751,0,0,1,60000000,4,0


In [35]:
CAPS = [0,10_000_000,20_000_000,40_000_000,60_000_000,80_000_000,100_000_000,200_000_000,500_000_000]
cap_to_idx = {cap:i for i,cap in enumerate(CAPS)}
data['fee_bin_idx'] = data['hoc_phi_ceiled'].map(cap_to_idx).astype('int8')   # feature số/ordered category
data['is_free']     = (data['hoc_phi_ceiled'] == 0).astype('int8')

In [37]:
y

array([491, 491, 491, ..., 850, 833, 797])

In [ ]:
classes, y_enc = np.unique(y, return_inverse=True)

X_train, X_valid, y_train, y_valid = train_test_split(
    data, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)

vc = pd.Series(y_train).value_counts()
cls_w = {int(k): float(1.0/np.sqrt(v)) for k, v in vc.items()}
w_train = pd.Series(y_train).map(cls_w).astype('float32').values
w_valid = pd.Series(y_valid).map(cls_w).fillna(float(np.mean(list(cls_w.values())))).astype('float32').values

d_train = lgb.Dataset(
    X_train, label=y_train, weight=w_train,
    categorical_feature=cat_cols, free_raw_data=False
)
d_valid = lgb.Dataset(
    X_valid, label=y_valid, weight=w_valid,   # có thể bỏ weight=... nếu không cần
    categorical_feature=cat_cols, reference=d_train, free_raw_data=False
)


In [38]:
params = {
    'objective': 'multiclass',
    'num_class': len(classes),
    'metric': 'multi_logloss',
    'learning_rate': 0.05,
    'num_leaves': 63,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 1,
    'verbose': -1,
    'seed': 42,
}
clf = lgb.train(
    params,
    d_train,
    num_boost_round=1000,
    valid_sets=[d_train, d_valid],
    valid_names=['train','valid'],
    callbacks=[lgb.early_stopping(150), lgb.log_evaluation(50)]
)

Training until validation scores don't improve for 150 rounds
[50]	train's multi_logloss: 33.0749	valid's multi_logloss: 33.0536
[100]	train's multi_logloss: 32.6595	valid's multi_logloss: 32.6262
[150]	train's multi_logloss: 33.7011	valid's multi_logloss: 33.732
Early stopping, best iteration is:
[1]	train's multi_logloss: 4.82171	valid's multi_logloss: 4.69209


In [39]:
p_valid = clf.predict(X_valid, num_iteration=getattr(clf, "best_iteration", None))

from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_fscore_support

num_class = p_valid.shape[1]
Y_true = label_binarize(y_valid, classes=np.arange(num_class))  # one-hot ground truth
thr = 0.05  # ngưỡng xác suất để chọn label
Y_pred = (p_valid >= thr).astype(int)

# micro/samples (samples = tính theo từng hàng rồi trung bình)
p_micro, r_micro, f_micro, _     = precision_recall_fscore_support(Y_true, Y_pred, average="micro",   zero_division=0)
p_samples, r_samples, f_samples, _ = precision_recall_fscore_support(Y_true, Y_pred, average="samples", zero_division=0)

print({
    "precision_micro_thr": p_micro, "recall_micro_thr": r_micro, "f1_micro_thr": f_micro,
    "precision_samples_thr": p_samples, "recall_samples_thr": r_samples, "f1_samples_thr": f_samples,
    "threshold": thr
})


{'precision_micro_thr': 0.27721786736344406, 'recall_micro_thr': 0.5445435659293658, 'f1_micro_thr': 0.3673991012903492, 'precision_samples_thr': 0.3428311016451063, 'recall_samples_thr': 0.5445435659293658, 'f1_samples_thr': 0.3918180160572926, 'threshold': 0.05}


## per-group classifier (LogisticRegression/MultinomialNB)

In [40]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [112]:
group_cols   = ['cong_lap','tinh_tp','nhom_nganh']
target_col   = 'truong_nganh_ptxt'
ohe_cols     = ['cong_lap','tinh_tp','nhom_nganh','hsg_subject',
                'ahld','dan_toc_thieu_so','haimuoi_huyen_ngheo_tnb',]

# CAPS = [0,10_000_000,20_000_000,40_000_000,60_000_000,80_000_000,100_000_000,200_000_000,500_000_000]
# cap_to_idx = {cap:i for i,cap in enumerate(CAPS)}
# def round_budget_series(s: pd.Series) -> pd.Series:
#     x = pd.to_numeric(s, errors='coerce').fillna(0).astype(float)
#     bins   = [-np.inf, 0, 10e6, 20e6, 40e6, 60e6, 80e6, 100e6, 200e6, np.inf]
#     labels = [0,        10e6,  20e6,  40e6,  60e6,  80e6,  100e6,  200e6, 500e6]
#     return pd.cut(x, bins=bins, labels=labels, right=True, include_lowest=True).astype(float).astype('int64')

# df: pd.DataFrame = df.copy()
# df['fee_bin_idx'] = round_budget_series(df['hoc_phi']).map(cap_to_idx).fillna(0).astype('int8')


In [113]:
for c in ['tinh_tp','hsg_subject', target_col]:
    df[c] = df[c].astype(str).replace({'nan':'UNK','None':'UNK'})

In [114]:
models = {}            
encoders = {}          
label_encoders = {}   
class_lists = {} 

for gkey, gdf in df.groupby(group_cols, dropna=False):
    # label encoding per group
    y_text = gdf[target_col].astype(str).values
    le = LabelEncoder()
    y = le.fit_transform(y_text)

    if np.unique(y).size <= 1:
        models[gkey] = None
        encoders[gkey] = None
        label_encoders[gkey] = le
        class_lists[gkey] = le.classes_.tolist()
        continue

    enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.float32)
    Xg = enc.fit_transform(gdf[ohe_cols].astype(str))

    clf = LogisticRegression(
        max_iter=1000,
        solver='lbfgs',
        multi_class='auto',
        class_weight='balanced',
        n_jobs=None
    )

    clf.fit(Xg, y)

    models[gkey] = clf
    encoders[gkey] = enc
    label_encoders[gkey] = le
    class_lists[gkey] = le.classes_.tolist()

len(models)

/var/folders/fx/pwxbtsr91db75yjrw7q39zf00000gp/T/ipykernel_3158/2956296568.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for gkey, gdf in df.groupby(group_cols, dropna=False):
/opt/miniconda3/envs/eda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/miniconda3/envs/eda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/miniconda3/envs/eda/lib

165

In [115]:
df

,cong_lap,tinh_tp,to_hop_mon,hoc_phi,hsg_1,hsg_2,hsg_3,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,truong_nganh_ptxt,nhom_nganh,hsg_subject,prio_any,fee_bin_idx
0,1,An Giang,UNK,14100000.0,Toán,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Toán,1,2
1,1,An Giang,UNK,14100000.0,Lý,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Lý,1,2
2,1,An Giang,UNK,14100000.0,Hoá,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Hoá,1,2
3,1,An Giang,UNK,14100000.0,Sinh,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Sinh,1,2
4,1,An Giang,UNK,14100000.0,Tin,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714,Tin,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40910,1,TP. Hồ Chí Minh,C01,62000000.0,0,0,0,0,0,1,SPK-7510303A-Ưu Tiên,751,0,1,5
40911,1,Bình Phước,C01,32000000.0,0,0,0,0,0,1,SPK-7510301BP-Ưu Tiên,751,0,1,3
40912,1,TP. Hồ Chí Minh,C02,32000000.0,0,0,0,0,0,1,SPK-7510401V-Ưu Tiên,751,0,1,3
40913,1,TP. Hồ Chí Minh,A01,49000000.0,0,0,0,0,0,1,SPK-7510205N-Ưu Tiên,751,0,1,4


In [116]:
cong_lap = [1,0]
tinh_tp = ["TP. Hồ Chí Minh","An Giang","Bình Dương","Cần Thơ","Vũng Tàu"]
hoc_phi = [0,10000000,20000000,40000000,60000000,80000000,100000000,200000000,500000000]
hsg_subject = ["Toán", "Lý", "Anh", "Văn", "Hoá", "Sinh", "Tin", "Sử", "Địa", "Tiếng Pháp", "Tiếng Trung", "Tiếng Nhật", "Tiếng Nga"]
nhom_nganh = [714, 734, 738, 742, 748, 751, 754, 762, 731, 722, 785, 764, 721, 746, 758,
772, 781, 732, 752, 784, 776, 744]
ahld = [0, 1]
dan_toc_thieu_so = [0, 1]
haimuoi_huyen_ngheo_tnb = [0, 1]
cap_to_idx = {cap:i for i,cap in enumerate(CAPS)}

test = pd.DataFrame({
    'cong_lap': np.random.choice(cong_lap, size=100),
    'tinh_tp': np.random.choice(tinh_tp, size=100),
    'hoc_phi': np.random.choice(hoc_phi, size=100),
    'nhom_nganh': np.random.choice(nhom_nganh, size=100),
    'hsg_subject': np.random.choice(hsg_subject, size=100),
    'ahld': np.random.choice(ahld, size=100),
    'dan_toc_thieu_so': np.random.choice(dan_toc_thieu_so, size=100),
    'haimuoi_huyen_ngheo_tnb': np.random.choice(haimuoi_huyen_ngheo_tnb, size=100),
})
test['hoc_phi_ceiled'] = test['hoc_phi'].copy()
import pandas as pd

# đảm bảo dtype
test = test.copy()
test['hsg_subject'] = test['hsg_subject'].fillna("0").astype(str)
for c in ['ahld', 'dan_toc_thieu_so', 'haimuoi_huyen_ngheo_tnb']:
    test[c] = pd.to_numeric(test[c], errors='coerce').fillna(0).astype(int)

rows = []

hsg_df = test[test['hsg_subject'] != "0"].copy()
hsg_df[['ahld', 'dan_toc_thieu_so', 'haimuoi_huyen_ngheo_tnb']] = 0
rows.append(hsg_df)

ahld_df = test[test['ahld'] == 1].copy()
ahld_df['hsg_subject'] = "0"
ahld_df[['dan_toc_thieu_so', 'haimuoi_huyen_ngheo_tnb']] = 0
rows.append(ahld_df)

dtts_df = test[test['dan_toc_thieu_so'] == 1].copy()
dtts_df['hsg_subject'] = "0"
dtts_df[['ahld', 'haimuoi_huyen_ngheo_tnb']] = 0
rows.append(dtts_df)

huyen_df = test[test['haimuoi_huyen_ngheo_tnb'] == 1].copy()
huyen_df['hsg_subject'] = "0"
huyen_df[['ahld', 'dan_toc_thieu_so']] = 0
rows.append(huyen_df)

none_df = test[
    (test['hsg_subject'] == "0") &
    (test['ahld'] == 0) &
    (test['dan_toc_thieu_so'] == 0) &
    (test['haimuoi_huyen_ngheo_tnb'] == 0)
].copy()
rows.append(none_df)

# Gộp & loại bỏ mọi dòng còn >1 ưu tiên
out = pd.concat(rows, ignore_index=True)

active_cnt = (
    (out['hsg_subject'] != "0").astype(int)
    + out['ahld'].astype(int)
    + out['dan_toc_thieu_so'].astype(int)
    + out['haimuoi_huyen_ngheo_tnb'].astype(int)
)
out = out[active_cnt <= 1].reset_index(drop=True)


out['fee_bin_idx'] = out['hoc_phi'].map(cap_to_idx).fillna(0).astype('int8')
out['is_free'] = (out['hoc_phi_ceiled'] == 0).astype('int8')
out



,cong_lap,tinh_tp,hoc_phi,nhom_nganh,hsg_subject,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,hoc_phi_ceiled,fee_bin_idx,is_free
0,0,Vũng Tàu,80000000,784,Hoá,0,0,0,80000000,5,0
1,0,An Giang,100000000,758,Tiếng Trung,0,0,0,100000000,6,0
2,1,Vũng Tàu,0,738,Anh,0,0,0,0,0,1
3,1,TP. Hồ Chí Minh,500000000,721,Tiếng Trung,0,0,0,500000000,8,0
4,1,An Giang,60000000,752,Tiếng Nga,0,0,0,60000000,4,0
...,...,...,...,...,...,...,...,...,...,...,...
232,1,Vũng Tàu,100000000,758,0,0,0,1,100000000,6,0
233,0,An Giang,10000000,762,0,0,0,1,10000000,1,0
234,1,TP. Hồ Chí Minh,0,732,0,0,0,1,0,0,1
235,0,Cần Thơ,20000000,721,0,0,0,1,20000000,2,0


In [117]:
out.columns

Index(['cong_lap', 'tinh_tp', 'hoc_phi', 'nhom_nganh', 'hsg_subject', 'ahld',
       'dan_toc_thieu_so', 'haimuoi_huyen_ngheo_tnb', 'hoc_phi_ceiled',
       'fee_bin_idx', 'is_free'],
      dtype='object')

In [118]:
def predict(df):
    records = []
    for idx, r in df.iterrows():
        gkey = tuple(r[c] for c in group_cols)
        clf = models.get(gkey)
        enc = encoders.get(gkey)
        le  = label_encoders.get(gkey)

        if le is None:
            records.append({
                'row_id': idx,
                'group': gkey,
                'status': 'unseen_group',
                'pred': None,
                'probs': None
            })
            continue

        if clf is None:
            # single-class group -> deterministic prediction
            only_label = le.classes_[0]
            records.append({
                'row_id': idx,
                'group': gkey,
                'status': 'single_class',
                'pred': only_label,
                'probs': {only_label: 1.0}
            })
            continue

        # one-hot transform with this group's encoder
        X1 = enc.transform(pd.DataFrame([r[ohe_cols].astype(str)]))
        if hasattr(clf, "predict_proba"):
            p = clf.predict_proba(X1)[0]
            labels = le.classes_
            items = list(zip(labels, p))
            items.sort(key=lambda x: -x[1])
            
            pred = items[0][0]
            probs = {lab: float(pr) for lab, pr in items}
        else:
            # NB always has predict_proba; this is a fallback
            yhat = clf.predict(X1)[0]
            pred = le.inverse_transform([yhat])[0]
            probs = None

        records.append({
            'row_id': idx,
            'group': gkey,
            'status': 'ok',
            'pred': pred,
            'probs': probs
        })

    return pd.DataFrame(records)
pred_df = predict(out)

In [119]:
print(pred_df)

     row_id                      group        status                     pred  \
0         0         (0, Vũng Tàu, 784)  unseen_group                     None   
1         1         (0, An Giang, 758)  unseen_group                     None   
2         2         (1, Vũng Tàu, 738)  unseen_group                     None   
3         3  (1, TP. Hồ Chí Minh, 721)            ok  IUH-7210404-TUYỂN THẲNG   
4         4         (1, An Giang, 752)  unseen_group                     None   
..      ...                        ...           ...                      ...   
232     232         (1, Vũng Tàu, 758)  unseen_group                     None   
233     233         (0, An Giang, 762)  unseen_group                     None   
234     234  (1, TP. Hồ Chí Minh, 732)            ok      SGD-7320201-Ưu Tiên   
235     235          (0, Cần Thơ, 721)  unseen_group                     None   
236     236  (0, TP. Hồ Chí Minh, 754)  unseen_group                     None   

                           

In [120]:
df_test = out.reset_index(drop=True).assign(row_id=lambda d: d.index)

full_out = (
    df_test
    .merge(pred_df, on='row_id', how='left', suffixes=('', '_pred'))
)
print(full_out)

     cong_lap          tinh_tp    hoc_phi  nhom_nganh  hsg_subject  ahld  \
0           0         Vũng Tàu   80000000         784          Hoá     0   
1           0         An Giang  100000000         758  Tiếng Trung     0   
2           1         Vũng Tàu          0         738          Anh     0   
3           1  TP. Hồ Chí Minh  500000000         721  Tiếng Trung     0   
4           1         An Giang   60000000         752    Tiếng Nga     0   
..        ...              ...        ...         ...          ...   ...   
232         1         Vũng Tàu  100000000         758            0     0   
233         0         An Giang   10000000         762            0     0   
234         1  TP. Hồ Chí Minh          0         732            0     0   
235         0          Cần Thơ   20000000         721            0     0   
236         0  TP. Hồ Chí Minh  100000000         754            0     0   

     dan_toc_thieu_so  haimuoi_huyen_ngheo_tnb  hoc_phi_ceiled  fee_bin_idx  \
0       

In [110]:
results = full_out[full_out["hoc_phi_ceiled"]<=20000000]

In [111]:
results

,cong_lap,tinh_tp,hoc_phi,nhom_nganh,hsg_subject,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,hoc_phi_ceiled,fee_bin_idx,is_free,row_id,group,status,pred,probs
0,0,Vũng Tàu,10000000,764,Tin,0,0,0,10000000,1,0,0,"(0, Vũng Tàu, 764)",unseen_group,None,None
2,1,TP. Hồ Chí Minh,0,744,Tiếng Pháp,0,0,0,0,0,1,2,"(1, TP. Hồ Chí Minh, 744)",ok,QST-7440122-Ưu Tiên,"{'QST-7440122-Ưu Tiên': 0.08915140642813726, '..."
3,1,Vũng Tàu,20000000,732,Tiếng Pháp,0,0,0,20000000,2,0,3,"(1, Vũng Tàu, 732)",unseen_group,None,None
8,1,Bình Dương,10000000,746,Tin,0,0,0,10000000,1,0,8,"(1, Bình Dương, 746)",single_class,TDM-7460101-TUYỂN THẲNG,{'TDM-7460101-TUYỂN THẲNG': 1.0}
9,1,Cần Thơ,0,772,Lý,0,0,0,0,0,1,9,"(1, Cần Thơ, 772)",single_class,TCT-7720203-TUYỂN THẲNG,{'TCT-7720203-TUYỂN THẲNG': 1.0}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,1,Bình Dương,0,722,0,0,0,1,0,0,1,243,"(1, Bình Dương, 722)",ok,DBD-7220201-TUYỂN THẲNG,"{'DBD-7220201-TUYỂN THẲNG': 0.25, 'TDM-7220201..."
246,0,Vũng Tàu,0,764,0,0,0,1,0,0,1,246,"(0, Vũng Tàu, 764)",unseen_group,None,None
248,1,TP. Hồ Chí Minh,20000000,738,0,0,0,1,20000000,2,0,248,"(1, TP. Hồ Chí Minh, 738)",ok,MBS-7380107C-TUYỂN THẲNG,{'MBS-7380107C-TUYỂN THẲNG': 0.053534624424316...
249,1,Cần Thơ,0,752,0,0,0,1,0,0,1,249,"(1, Cần Thơ, 752)",ok,TCT-7520103-TUYỂN THẲNG,{'TCT-7520103-TUYỂN THẲNG': 0.0833333333333333...


In [88]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40915 entries, 0 to 40914
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Công lập (1/0)           40915 non-null  int64  
 1   Tỉnh/TP                  40915 non-null  object 
 2   Tổ hợp môn               3393 non-null   object 
 3   Học phí                  40553 non-null  float64
 4   hsg_1                    40915 non-null  object 
 5   hsg_2                    40915 non-null  object 
 6   hsg_3                    40915 non-null  object 
 7   ahld                     40915 non-null  int64  
 8   dan_toc_thieu_so         40915 non-null  int64  
 9   haimuoi_huyen_ngheo_tnb  40915 non-null  int64  
 10  truong-nganh-ptxt        40915 non-null  object 
 11  nhom_nganh               40915 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 3.7+ MB


In [87]:
df1[(df1["Công lập (1/0)"] == 1) & (df1["Tỉnh/TP"] == "TP. Hồ Chí Minh") & (df1["Học phí"] == 0.0) & (df1["nhom_nganh"] == 754) & (df1["hsg_1"] == "Văn") ]

,Công lập (1/0),Tỉnh/TP,Tổ hợp môn,Học phí,hsg_1,hsg_2,hsg_3,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,truong-nganh-ptxt,nhom_nganh


In [121]:
import joblib, sklearn, sys
from datetime import datetime

bundle = {
    "models": models,
    "encoders": encoders,
    "label_encoders": label_encoders,
    "class_lists": class_lists,
    "meta": {
        "created_at": datetime.now().isoformat(timespec="seconds"),
        "sklearn_version": sklearn.__version__,
        "python_version": sys.version.split()[0],
        "group_cols": group_cols,
        "ohe_cols": ohe_cols,
        "target_col": target_col,
    },
}

save_path = "../../models/l1_model.joblib"
joblib.dump(bundle, save_path, compress=3)
save_path


'../../models/l1_model.joblib'